In [ ]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
import h5py
import tidynamics
import subprocess
import os, os.path
import time

In [ ]:
cd ~/compute/ABP_lammps/probe_campaign_2/

In [ ]:
%run abp_util

In [ ]:
params = {
    'tau_r': 10,
    'v0': 0,
    'hfile': 'abp.h5',
    'sigma_probe': 3,
    'add_force': 0,
    'sampling': 16000,
    'dump_every': 100,
    'L': 30,
    'rho': 0.1,
    'bath_dump_every': 100,
    'bath_hfile': 'active_bath.h5'
}

In [ ]:
vary_k, vary_v = 'v0', [0, 10, 20]

In [ ]:
t0 = time.time()

res = {}

for i, val in enumerate(vary_v):
    p = params.copy()
    p.update({vary_k: val, 'hfile': f'abp_{i}.h5', 'bath_hfile': f'active_bath_{i}.h5'})
    print(p)
    parameter_file = write_input(**p)
    with open(f'abp_{i}.in', 'w') as f:
        f.write(parameter_file)
    # here, run the simulation
    subprocess.run([os.path.join(os.path.expanduser('~'), 'code', 'lammps', 'build_fix_active', 'lmp'), '-i', f'abp_{i}.in'])
    #with h5py.File('abp.h5', 'r') as a:
        # load data
    

for i, val in enumerate(vary_v):

    with h5py.File(p['hfile'], 'r') as a:
        L = a['particles/probe/box/edges/value'][0]
        r = a['particles/probe/position/value'][:,0,:]
        im = a['particles/probe/image/value'][:,0,:]
        r += im*L[None,:]
        v = a['particles/probe/velocity/value'][:,0,:]
        r_t = a['particles/probe/position/time'][:]
        v_t = a['particles/probe/velocity/time'][:]

    r_t -= r_t[0]
    v_t -= v_t[0]
    u = v / np.sqrt(np.sum(v**2, axis=1))[:,None]

    d = {}
    d['msd'] = tidynamics.msd(r)
    d['vacf'] = tidynamics.acf(v)
    d['uacf'] = tidynamics.acf(u)
    d['x'] = r[:,0]

    res[val] = d

    print(val)

print("elapsed time:", time.time()-t0)

In [ ]:
plt.figure()

for val in vary_v:
    d = res[val]
    plt.plot(r_t, d['msd'], label=vary_k+' '+str(val))

plt.plot(r_t, 4/3*r_t, label='passive diff')

plt.legend()

In [ ]:
plt.figure()

for val in vary_v:
    d = res[val]
    plt.plot(v_t, d['vacf'])

plt.xlim(0, 10)